<a href="https://colab.research.google.com/github/drfperez/utilities/blob/main/PDF2Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# =====================================================
# 🧮 OCR MATEMÀTIC ÒPTIM — FINAL DEFINITIU (COLAB)
# =====================================================

dpi = 350
crear_pdf_searchable = True
IDIOMES = "cat+spa+eng"

!apt-get update -qq
!apt-get install -y -qq \
    poppler-utils tesseract-ocr \
    tesseract-ocr-cat tesseract-ocr-spa tesseract-ocr-eng \
    ghostscript pngquant

!pip install -q pytesseract pdf2image python-docx ocrmypdf Pillow numpy opencv-python-headless

import os, re, cv2, numpy as np
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
import pytesseract, ocrmypdf
from docx import Document

TESS_CONFIG = "--oem 1 --psm 6 -c preserve_interword_spaces=1"

def deskew(img):
    g = cv2.bitwise_not(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    coords = np.column_stack(np.where(g > 0))
    if len(coords) < 30:
        return img
    angle = cv2.minAreaRect(coords)[-1]
    angle = -(90 + angle) if angle < -45 else -angle
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(img, M, (w, h),
        flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

def preprocess(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = cv2.fastNlMeansDenoising(g, None, 10, 7, 21)
    clahe = cv2.createCLAHE(2.5, (8,8))
    g = clahe.apply(g)
    t = cv2.adaptiveThreshold(
        g, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 31, 9
    )
    return cv2.cvtColor(t, cv2.COLOR_GRAY2RGB)

def clean_math_text(t):
    t = re.sub(r'[\x00-\x1f\x7f]', '', t)
    t = re.sub(r'[ \t]+', ' ', t)
    fixes = {"<=":"≤", ">=":"≥", "!=":"≠", " x ":" × ", " / ":" ÷ "}
    for k,v in fixes.items():
        t = t.replace(k,v)
    return t.strip()

uploaded = files.upload()
if not uploaded:
    raise SystemExit("❌ Cap PDF seleccionat")

pdf_path = next(iter(uploaded))
base = os.path.splitext(pdf_path)[0]

images = convert_from_path(pdf_path, dpi=dpi)
doc = Document()

for i, img in enumerate(images):
    print(f"🧮 OCR pàgina {i+1}/{len(images)}")
    cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    cv = preprocess(deskew(cv))

    text = pytesseract.image_to_string(
        Image.fromarray(cv),
        lang=IDIOMES,
        config=TESS_CONFIG
    )

    doc.add_heading(f"Pàgina {i+1}", level=2)
    doc.add_paragraph(clean_math_text(text))
    if i < len(images)-1:
        doc.add_page_break()

docx_out = f"OCR_MATES_{base}.docx"
doc.save(docx_out)
files.download(docx_out)

if crear_pdf_searchable:
    pdf_out = f"OCR_MATES_searchable_{base}.pdf"
    ocrmypdf.ocr(
        pdf_path,
        pdf_out,
        language=IDIOMES,
        deskew=True,
        optimize=3,
        skip_text=True,            # ✅ CLAU
        oversample=300,
        tagged_pdf_mode="ignore"
    )
    files.download(pdf_out)

print("✅ OCR matemàtic FINAL — sense errors ni inflat")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:

# =====================================================
# 🧮 OCR MATEMÀTIC POTENT — 1 CEL·LA (GOOGLE COLAB)
# =====================================================

# ---------- CONFIGURACIÓ ----------
dpi = 450            # clau per símbols petits
upsample = 2         # millora grecs i operadors
crear_pdf_searchable = True

# ---------- INSTAL·LACIÓ ----------
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr \
    tesseract-ocr-cat tesseract-ocr-spa tesseract-ocr-eng ghostscript
!pip install -q pytesseract pdf2image python-docx easyocr \
    opencv-python-headless ocrmypdf Pillow numpy

# ---------- IMPORTS ----------
import os, re, cv2, numpy as np
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
import pytesseract, easyocr, ocrmypdf
from docx import Document

# ---------- TESSERACT (CATALÀ + MATES) ----------
TESS_LANG = "cat+spa+eng"
TESS_CONFIG = (
    "--oem 1 --psm 6 "
    "-c preserve_interword_spaces=1 "
    "-c tessedit_char_whitelist="
    "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    "αβγδεζηθικλμνξοπρστυφχψω"
    "ΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩ"
    "+-−×÷=≠≈≤≥<>^*/()[]{}|"
    "∑∏∫√∞°±µπλΔΩ"
    ".,;:!?@#%€$\\ "
)

# ---------- FUNCIONS ----------
def pil_to_cv(img):
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

def cv_to_pil(img):
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

def deskew(img):
    gray = cv2.bitwise_not(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    coords = np.column_stack(np.where(gray > 0))
    if len(coords) < 20:
        return img
    angle = cv2.minAreaRect(coords)[-1]
    angle = -(90 + angle) if angle < -45 else -angle
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(img, M, (w, h),
        flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

def preprocess_math(img, up=2):
    if up > 1:
        img = cv2.resize(img, None, fx=up, fy=up,
                         interpolation=cv2.INTER_CUBIC)
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = cv2.fastNlMeansDenoising(g, None, 12, 7, 21)
    clahe = cv2.createCLAHE(3.0, (8,8))
    g = clahe.apply(g)
    t = cv2.adaptiveThreshold(
        g, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 35, 11
    )
    return cv2.cvtColor(t, cv2.COLOR_GRAY2BGR)

def clean_math_text(t):
    t = t.replace("\r", "\n")
    t = re.sub(r'-(\n)', '', t)
    t = re.sub(r'[ \t]+', ' ', t)
    t = re.sub(r'\n{3,}', '\n\n', t)
    t = re.sub(r'[\x00-\x1f\x7f]', '', t)

    fixes = {
        "<=": "≤", ">=": "≥", "!=": "≠",
        " - ": " − ", " x ": " × ", " / ": " ÷ ",
        " O ": " 0 ", " l ": " 1 "
    }
    for k, v in fixes.items():
        t = t.replace(k, v)
    return t.strip()

# ---------- PUJAR PDF ----------
uploaded = files.upload()
pdf_path = next(iter(uploaded))

# ---------- OCR ----------
images = convert_from_path(pdf_path, dpi=dpi)
reader = easyocr.Reader(["en"], gpu=False)  # ⚠️ només EN (correcte)
doc = Document()

for i, img in enumerate(images):
    print(f"🧮 Pàgina {i+1}/{len(images)}")
    cv = preprocess_math(deskew(pil_to_cv(img)), upsample)

    t_text = pytesseract.image_to_string(
        cv_to_pil(cv),
        lang=TESS_LANG,
        config=TESS_CONFIG
    )

    e_text = "\n".join(reader.readtext(cv, detail=0))

    final = e_text if len(e_text) > len(t_text)*0.85 else t_text
    final = clean_math_text(final)

    doc.add_heading(f"Pàgina {i+1}", level=2)
    doc.add_paragraph(final)
    if i < len(images)-1:
        doc.add_page_break()

# ---------- DESAR ----------
base = os.path.splitext(pdf_path)[0]
docx_out = f"OCR_MATES_{base}.docx"
doc.save(docx_out)
files.download(docx_out)

if crear_pdf_searchable:
    pdf_out = f"OCR_MATES_searchable_{base}.pdf"
    ocrmypdf.ocr(
        pdf_path,
        pdf_out,
        language="cat",
        deskew=True,
        force_ocr=True
    )
    files.download(pdf_out)

print("✅ OCR matemàtic complet")

JBIG2                 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/0 -:--:--

Possible reasons for this include:
--deskew was issued, causing transcoding.
--force-ocr (or --mode force) was issued, causing transcoding.
The optional dependency 'jbig2' was not found, so some image optimizations could not be attempted.
The optional dependency 'pngquant' was not found, so some image optimizations could not be attempted.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ OCR matemàtic complet


In [ ]:
# =====================================================
# 📄 PDF → DOCX EDITABLE (OCR MATEMÀTIC POTENT)
# 1 CEL·LA · GOOGLE COLAB
# =====================================================

# ---------- CONFIGURACIÓ ----------
dpi = 400          # suficient si el PDF ja té OCR
idioma = "cat+spa+eng"

# ---------- INSTAL·LACIÓ ----------
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr \
    tesseract-ocr-cat tesseract-ocr-spa tesseract-ocr-eng
!pip install -q pytesseract pdf2image python-docx Pillow numpy

# ---------- IMPORTS ----------
import os, re, numpy as np
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
import pytesseract
from docx import Document

# ---------- CONFIG TESSERACT (MATES) ----------
TESS_CONFIG = (
    "--oem 1 --psm 6 "
    "-c preserve_interword_spaces=1 "
    "-c tessedit_char_whitelist="
    "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    "αβγδεζηθικλμνξοπρστυφχψω"
    "ΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩ"
    "+-−×÷=≠≈≤≥<>^*/()[]{}|"
    "∑∏∫√∞°±µπλΔΩ"
    ".,;:!?@#%€$\\ "
)

# ---------- NETEJA DE TEXT ----------
def clean_text(t):
    t = t.replace("\r", "\n")
    t = re.sub(r'-(\n)', '', t)
    t = re.sub(r'[ \t]+', ' ', t)
    t = re.sub(r'\n{3,}', '\n\n', t)
    t = re.sub(r'[\x00-\x1f\x7f]', '', t)

    fixes = {
        "<=": "≤", ">=": "≥", "!=": "≠",
        " x ": " × ", " / ": " ÷ ",
        " O ": " 0 ", " l ": " 1 "
    }
    for k, v in fixes.items():
        t = t.replace(k, v)
    return t.strip()

# ---------- PUJAR PDF ----------
uploaded = files.upload()
pdf_path = next(iter(uploaded))
base = os.path.splitext(pdf_path)[0]

# ---------- OCR → DOCX ----------
print("📸 Convertint PDF a imatges…")
images = convert_from_path(pdf_path, dpi=dpi)
doc = Document()

for i, img in enumerate(images):
    print(f"🧮 Processant pàgina {i+1}/{len(images)}")
    text = pytesseract.image_to_string(
        img,
        lang=idioma,
        config=TESS_CONFIG
    )
    text = clean_text(text)

    doc.add_heading(f"Pàgina {i+1}", level=2)
    doc.add_paragraph(text)

    if i < len(images) - 1:
        doc.add_page_break()

# ---------- DESAR ----------
out_docx = f"{base}_editable.docx"
doc.save(out_docx)
files.download(out_docx)

print("✅ DOCX editable creat correctament")

In [ ]:
# @title 📄 OCR per a PDF Escanejats (Imatges)
idioma_seleccionat = "Català" # @param ["Català", "Castellà", "Anglès", "Mix (Català + Castellà)"]

import re
import pytesseract
from pdf2image import convert_from_path
from docx import Document
from google.colab import files

# 1. INSTAL·LACIÓ (Més ràpida si ja s'ha executat abans)
print("⚙️ Preparant eines...")
!sudo apt-get update > /dev/null
!sudo apt-get install tesseract-ocr tesseract-ocr-cat tesseract-ocr-spa tesseract-ocr-eng poppler-utils > /dev/null
!pip install pytesseract pdf2image python-docx > /dev/null

lang_codes = {"Català": "cat", "Castellà": "spa", "Anglès": "eng", "Mix (Català + Castellà)": "cat+spa"}
lang_code = lang_codes[idioma_seleccionat]

def clean_text(text):
    # Elimina caràcters de control no vàlids per a XML/Word
    return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', text)

# 2. EXECUCIÓ
uploaded = files.upload()

if uploaded:
    pdf_filename = next(iter(uploaded))
    try:
        # Augmentem la resolució (dpi=300) per a millor lectura d'imatges
        print(f"📸 Convertint pàgines a alta resolució...")
        images = convert_from_path(pdf_filename, dpi=300)
        doc = Document()

        for i, image in enumerate(images):
            print(f"🔍 Llegint pàgina {i+1}...")
            # '--psm 1' detecta automàticament l'orientació i segments de text
            raw_text = pytesseract.image_to_string(image, lang=lang_code, config='--psm 1')

            safe_text = clean_text(raw_text)
            doc.add_heading(f'Pàgina {i+1}', level=2)
            doc.add_paragraph(safe_text)

            if i < len(images) - 1:
                doc.add_page_break()

        out_name = f"OCR_{pdf_filename.replace('.pdf', '.docx')}"
        doc.save(out_name)
        print(f"\n✅ Enllestit! Descarregant...")
        files.download(out_name)

    except Exception as e:
        print(f"\n❌ Error: {e}")

⚙️ Preparant eines...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Saving digitalproblemes.pdf to digitalproblemes (1).pdf
📸 Convertint pàgines a alta resolució...
🔍 Llegint pàgina 1...
🔍 Llegint pàgina 2...
🔍 Llegint pàgina 3...
🔍 Llegint pàgina 4...
🔍 Llegint pàgina 5...
🔍 Llegint pàgina 6...

✅ Enllestit! Descarregant...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>